Импортируем библиотеку и данные

In [59]:
import pandas as pd

events_df = pd.read_csv('7_4_Events.csv')
purchase_df = pd.read_csv('7_4_Purchase.csv')

Отфильтруем полученнные данные в соответствии с заданием и произведем преобразование типа и даты 

In [60]:
cond = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
registered = events_df[cond]['user_id'].to_list()
events_df = events_df[events_df.user_id.isin(registered)]
events_df.start_time = pd.to_datetime(events_df.start_time, format='%Y-%m-%dT%H:%M:%S')

purchase_df['event_type'] = 'purchase'
purchase_df = purchase_df[purchase_df.user_id.isin(registered)]
purchase_df.event_datetime = pd.to_datetime(purchase_df.event_datetime, format='%Y-%m-%dT%H:%M:%S')

Переименуем колонки и произведем объединение

In [61]:
events_df = events_df.rename(columns={'id':'event_id'})
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})

total_events_df = pd.concat([events_df,purchase_df],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

Распределим пользователей по группам, в зависимости от выбора уровня сложности бесплатных тренировок. Сформируем датафреймы по группам пользователей с датой регистрации и с выбором уровня сложности, а также произведем замену названия колонок даты и времени события. Произведем проверки единичности событий.

In [62]:
users_with_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']['user_id'].unique()
users_with_easy_level_df_registration = total_events_df[(total_events_df['event_type'] == 'registration') & (total_events_df['user_id'].isin(users_with_easy_level))]
users_with_easy_level_df_registration = users_with_easy_level_df_registration[['user_id','start_time']].rename(columns={'start_time':'registration_time'})
print('Проверка единичности события регистрации "easy":', (users_with_easy_level_df_registration['user_id'].value_counts().mean()) == 1)
users_with_easy_level_df_level_choice = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(users_with_easy_level))]
users_with_easy_level_df_level_choice = users_with_easy_level_df_level_choice[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
print('Проверка единичности события выбора уровня сложности "easy":', (users_with_easy_level_df_level_choice['user_id'].value_counts().mean()) == 1)

users_with_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']['user_id'].unique()
users_with_medium_level_df_registration = total_events_df[(total_events_df['event_type'] == 'registration') & (total_events_df['user_id'].isin(users_with_medium_level))]
users_with_medium_level_df_registration = users_with_medium_level_df_registration[['user_id','start_time']].rename(columns={'start_time':'registration_time'})
print('Проверка единичности события регистрации "medium":', (users_with_medium_level_df_registration['user_id'].value_counts().mean()) == 1)
users_with_medium_level_df_level_choice = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(users_with_medium_level))]
users_with_medium_level_df_level_choice = users_with_medium_level_df_level_choice[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
print('Проверка единичности события выбора уровня сложности "medium":', (users_with_medium_level_df_level_choice['user_id'].value_counts().mean()) == 1)

users_with_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']['user_id'].unique()
users_with_hard_level_df_registration = total_events_df[(total_events_df['event_type'] == 'registration') & (total_events_df['user_id'].isin(users_with_hard_level))]
users_with_hard_level_df_registration = users_with_hard_level_df_registration[['user_id','start_time']].rename(columns={'start_time':'registration_time'})
print('Проверка единичности события регистрации "hard":', (users_with_hard_level_df_registration['user_id'].value_counts().mean()) == 1)
users_with_hard_level_df_level_choice = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(users_with_hard_level))]
users_with_hard_level_df_level_choice = users_with_hard_level_df_level_choice[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
print('Проверка единичности события выбора уровня сложности "hard":', (users_with_hard_level_df_level_choice['user_id'].value_counts().mean()) == 1)


Проверка единичности события регистрации "easy": True
Проверка единичности события выбора уровня сложности "easy": True
Проверка единичности события регистрации "medium": True
Проверка единичности события выбора уровня сложности "medium": True
Проверка единичности события регистрации "hard": True
Проверка единичности события выбора уровня сложности "hard": True


Посчитаем для каждого уровня сложности бесплатных тренировок количество и процент оплативших пользователей, а также временной промежуток между регистрацией и оплатой, выбором уровня сложности и оплатой.

Уровень сложности 'easy':

In [63]:
count_of_users_with_easy_level = len(users_with_easy_level)
purchase_df_of_users_with_easy_level = purchase_df[purchase_df['user_id'].isin(users_with_easy_level)]
percent_of_users_with_easy_level = purchase_df_of_users_with_easy_level['user_id'].nunique()/count_of_users_with_easy_level
print ('Процент оплативших пользователей, выбравших уровень сложности "easy": {:.2%}'.format(percent_of_users_with_easy_level))
print ()

purchase_df_of_users_with_easy_level_2 = purchase_df_of_users_with_easy_level[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})

merged_df_of_users_with_easy_level_registration = purchase_df_of_users_with_easy_level_2.merge(users_with_easy_level_df_registration,on='user_id',how='inner')
merged_df_of_users_with_easy_level_registration['timedelta'] = merged_df_of_users_with_easy_level_registration['purchase_time'] - merged_df_of_users_with_easy_level_registration['registration_time']
mean_time_of_users_with_easy_level_registration = merged_df_of_users_with_easy_level_registration['timedelta'].mean()
print ('Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "easy": {}'.format(mean_time_of_users_with_easy_level_registration))
print ('Характеристики времени:')
print (merged_df_of_users_with_easy_level_registration['timedelta'].describe())
print ()
merged_df_of_users_with_easy_level_level_choice = purchase_df_of_users_with_easy_level_2.merge(users_with_easy_level_df_level_choice,on='user_id',how='inner')
merged_df_of_users_with_easy_level_level_choice['timedelta'] = merged_df_of_users_with_easy_level_level_choice['purchase_time'] - merged_df_of_users_with_easy_level_level_choice['level_choice_time']
mean_time_of_users_with_easy_level_level_choice = merged_df_of_users_with_easy_level_level_choice['timedelta'].mean()
print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "easy": {}'.format(mean_time_of_users_with_easy_level_level_choice))
print ('Характеристики времени:')
print (merged_df_of_users_with_easy_level_level_choice['timedelta'].describe())
print ()

Процент оплативших пользователей, выбравших уровень сложности "easy": 7.72%

Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "easy": 3 days 22:10:23.211640211
Характеристики времени:
count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "easy": 3 days 14:58:52.941798941
Характеристики времени:
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype:

Уровень сложности 'medium':

In [64]:
count_of_users_with_medium_level = len(users_with_medium_level)
purchase_df_of_users_with_medium_level = purchase_df[purchase_df['user_id'].isin(users_with_medium_level)]
percent_of_users_with_medium_level = purchase_df_of_users_with_medium_level['user_id'].nunique()/count_of_users_with_medium_level
print ('Процент оплативших пользователей, выбравших уровень сложности "medium": {:.2%}'.format(percent_of_users_with_medium_level))
print ()

purchase_df_of_users_with_medium_level_2 = purchase_df_of_users_with_medium_level[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})

merged_df_of_users_with_medium_level_registration = purchase_df_of_users_with_medium_level_2.merge(users_with_medium_level_df_registration,on='user_id',how='inner')
merged_df_of_users_with_medium_level_registration['timedelta'] = merged_df_of_users_with_medium_level_registration['purchase_time'] - merged_df_of_users_with_medium_level_registration['registration_time']
mean_time_of_users_with_medium_level_registration = merged_df_of_users_with_medium_level_registration['timedelta'].mean()
print ('Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "medium": {}'.format(mean_time_of_users_with_medium_level_registration))
print ('Характеристики времени:')
print (merged_df_of_users_with_medium_level_registration['timedelta'].describe())
print ()
merged_df_of_users_with_medium_level_level_choice = purchase_df_of_users_with_medium_level_2.merge(users_with_medium_level_df_level_choice,on='user_id',how='inner')
merged_df_of_users_with_medium_level_level_choice['timedelta'] = merged_df_of_users_with_medium_level_level_choice['purchase_time'] - merged_df_of_users_with_medium_level_level_choice['level_choice_time']
mean_time_of_users_with_medium_level_level_choice = merged_df_of_users_with_medium_level_level_choice['timedelta'].mean()
print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "medium": {}'.format(mean_time_of_users_with_medium_level_level_choice))
print ('Характеристики времени:')
print (merged_df_of_users_with_medium_level_level_choice['timedelta'].describe())
print () 

Процент оплативших пользователей, выбравших уровень сложности "medium": 20.86%

Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "medium": 4 days 06:12:06.576883384
Характеристики времени:
count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "medium": 3 days 23:14:13.165118679
Характеристики времени:
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta,

Уровень сложности 'hard':

In [65]:
count_of_users_with_hard_level = len(users_with_hard_level)
purchase_df_of_users_with_hard_level = purchase_df[purchase_df['user_id'].isin(users_with_hard_level)]
percent_of_users_with_hard_level = purchase_df_of_users_with_hard_level['user_id'].nunique()/count_of_users_with_hard_level
print ('Процент оплативших пользователей, выбравших уровень сложности "hard": {:.2%}'.format(percent_of_users_with_hard_level))
print ()

purchase_df_of_users_with_hard_level_2 = purchase_df_of_users_with_hard_level[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})

merged_df_of_users_with_hard_level_registration = purchase_df_of_users_with_hard_level_2.merge(users_with_hard_level_df_registration,on='user_id',how='inner')
merged_df_of_users_with_hard_level_registration['timedelta'] = merged_df_of_users_with_hard_level_registration['purchase_time'] - merged_df_of_users_with_hard_level_registration['registration_time']
mean_time_of_users_with_hard_level_registration = merged_df_of_users_with_hard_level_registration['timedelta'].mean()
print ('Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "hard": {}'.format(mean_time_of_users_with_hard_level_registration))
print ('Характеристики времени:')
print (merged_df_of_users_with_hard_level_registration['timedelta'].describe())
print ()
merged_df_of_users_with_hard_level_level_choice = purchase_df_of_users_with_hard_level_2.merge(users_with_hard_level_df_level_choice,on='user_id',how='inner')
merged_df_of_users_with_hard_level_level_choice['timedelta'] = merged_df_of_users_with_hard_level_level_choice['purchase_time'] - merged_df_of_users_with_hard_level_level_choice['level_choice_time']
mean_time_of_users_with_hard_level_level_choice = merged_df_of_users_with_hard_level_level_choice['timedelta'].mean()
print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "hard": {}'.format(mean_time_of_users_with_hard_level_level_choice))
print ('Характеристики времени:')
print (merged_df_of_users_with_hard_level_level_choice['timedelta'].describe())
print ()

Процент оплативших пользователей, выбравших уровень сложности "hard": 35.39%

Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "hard": 3 days 14:55:19.257918552
Характеристики времени:
count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object

Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "hard": 3 days 07:20:41.420814479
Характеристики времени:
count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype

### АНАЛИЗ ПОЛУЧЕННЫХ ДАННЫХ И ВЫВОДЫ:

Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%
Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%
Процент оплативших пользователей, выбравших уровень сложности hard: 35.39%
Следовательно, с возрастанием уровня сложности тренировки возрастает процент оплативших пользователей.

Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "easy": 3 days 22:10:23.
Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "medium": 4 days 06:12:06.
Среднее время между регистрацией и оплатой для пользователей, выбравших уровень сложности "hard": 3 days 14:55:19.
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "easy": 3 days 14:58:52.
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "medium": 3 days 23:14:13.
Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности "hard": 3 days 07:20:41.

Наибольшее время между событием регистрации / выбора уровня сложности и моментом первой оплаты, тратят пользователи, которые выбрали уровень сложности medium.
Пользователи, выбравшие уровень сложности easy, тратят времени незначительно меньшеб чем которые выбрали уровень сложности medium(разница порядка 10%).
Меньше всего времени на принятие решение о покупке тратят пользователи, выбравшие уровень сложности hard.

Таким образом, по результату изучения датасета можно сделать вывод, что больше всего совершали оплату пользователи, которые завершали обучение и выбирали высокий уровень сложности тренировки. Также данные пользователи тратили меньше времени на обдумывание оплаты. Следовательно, на данная группая пользователей является "ключевым клиентом" приложения, так как именно она больше всего влияет на выручку.